<a href="https://colab.research.google.com/github/minnji88/NLP-study/blob/main/%5BDay04%5D_seq2seq_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# 인재양성 실무교육프로그램: Generative Bot based on Seq2Seq model

Primary TA: 이영준

TA's E-mail: passing2961@gmail.com


본 실습에서는 sequence-to-sequence (seq2seq) 모델을 이용하여 일상생활 대화가 가능한 챗봇을 개발한다. 또한, seq2seq 모델을 학습시키기 위해 12,000 쌍으로 이루어진 일상생활 대화 데이터를 사용할 것이다.

## Introduction

[seq2seq]: https://arxiv.org/abs/1409.3215
[Neural Conversational Model]: https://arxiv.org/pdf/1506.05869.pdf

Conversational Agent 라고도 불리는 챗봇은 기계가 자연어를 이용해 인간과 소통하기 위한 시스템이다. 대표적으로 잘 알려진 챗봇으로는 Google 의 Google Assistant 과 Apple 의 Siri 가 있다. 이렇듯 기존에 상용화된 챗봇들은 대부분 사용자의 요청하는 일을 처리하기 위함이 가장 큰 목적이다. 예를 들면, 사용자가 "오늘 날씨를 알려줘?" 라고 질문하였을 때, 챗봇은 오늘에 해당하는 날씨 정보를 가져와서 사용자에게 "오늘은 맑아" 라고 응답한다. 이렇게 상용화된 챗봇들은 주로 응답을 생성할 때, 미리 정의한 규칙이나 템플릿 기반으로 응답을 생성한다. 그렇다 보니, 실제로 사용자 입장에서는 챗봇이 생성하는 응답이 인간이 생성하는 응답처럼 느껴지지 않는다. 즉, 생성되는 응답이 비슷한 구조를 지니고 있게 된다. 그러므로, 챗봇에 대한 사용자의 참여 (engagement)가 떨어지게 되고 사용자에게 불안감 (frustration)을 형성시킬 수 있다. 이러한 이유로, 최근에는 end-to-end 방식의 딥러닝 모델을 이용하여 응답을 생성하는 연구들이 많이 이루어지고 있다. 대표적인 연구로는 Google 의 [Neural Conversational Model] 이 있다. 해당 모델은 인코더-디코더 구조인 [seq2seq 모델]이다. 따라서, 본 실습에서는 PyTorch 버전의 어텐션 기반의 seq2seq 모델을 개발할 것이다.



**Types of Chatbot Systems**

| Type  | Purpose | Characteristics |
| :------------ | :----------- | :------------------- |
| Task-oriented dialog systems | To complete specific tasks | Use hard-coded templates and rules |
| Non-task or open-domain chatbots | To imitate human conversation | Use the end-to-end deep learning model |

![chatbot](https://github.com/passing2961/KEMC/blob/master/chatbot.png?raw=true)


**Example**: 
<pre>
<code>
문장을 입력하세요: 안녕
Bot: 안녕하시어요.
문장을 입력하세요: 사랑해
Bot: 하늘만큼 땅만큼 사랑하아요.
문장을 입력하세요: 배고파
Bot: 뭐 좀 챙기어들시어요.
</code>
</pre>

## Key Points

- pynori 형태소 분석기를 이용한 한국어 대화 데이터 토크나이징
- Luong attention mechanism 기반 seq2seq 모델 구현
- seq2seq 모델의 학습 및 평가

## Acknowledgement

[chatbot 튜토리얼]: https://pytorch.org/tutorials/beginner/chatbot_tutorial.html#
[한국어 대화 데이터]: https://github.com/songys/Chatbot_data

- 본 실습 코드는 PyTorch 에서 제공하는 [chatbot 튜토리얼]을 참고하였습니다.
- seq2seq 모델의 학습을 위해 사용한 [한국어 대화 데이터]를 사용하였습니다.

## Step 0: Connect to Google drive

- 학습 도중 checkpoint 를 저장해서 나중에 불러와서 사용하고 싶은 경우에 필요하다.

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Step 1: Import modules

In [ ]:
# 라이브러리 호출
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import re
import random
import os
import csv
import pickle as pc
import collections
import numpy as np

# reproducibility 를 위함
torch.manual_seed(470)
torch.cuda.manual_seed(470)

# torch 버전 확인
print("Pytorch Version: ", torch.__version__)

# GPU 사용 가능한지 여부 확인
if torch.cuda.is_available():
    
    # PyTorch 에게 GPU 사용할거라고 알려주기
    device = torch.device("cuda")
    
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

Pytorch Version:  1.4.0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## Step 2: Configure the experiments

- 모델의 실험(학습 및 평가)을 필요한 파라미터 및 인자 설정
    - Hyperparameter: hidden unit size, vocabulary size, max length, dropout rate 등
    - Argument: file directory 등

In [ ]:
# 데이터, 모델 위치
data_dir = '/content/gdrive/My Drive/Colab Notebooks/data/ChatbotData.csv'

dirpath = '/content/gdrive/My Drive/Colab Notebooks/model/'
if not os.path.exists(dirpath):
    os.makedirs(dirpath)
    
WORD_DICT_DIR = '/content/gdrive/My Drive/Colab Notebooks/data/word2idx'
THRESHOLD = 40000
MAX_LEN = 25

attn_model = 'dot'
enc_hidden_size = 200
dec_hidden_size = 400
encoder_n_layers = 1
decoder_n_layers = 1
dropout = 0.1
batch_size = 32
max_epochs = 10

clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.001
max_patience = 5
best_loss = 100000.0

LOG_INTERVAL = 100

## Step 3: Data preparation

모델의 학습을 위해 데이터를 준비하는 단계로써, 다음의 과정을 거친다.
    
   - Load data
   - Tokenization
   - Build vocab

### Step 3-1: Load data

- **"ChatbotData.csv"** 파일 load
- **`[utterance, response]`** pair 의 형태로 재구성

In [ ]:
# pair data load
pair_data = list()

f = open(data_dir, 'r', encoding='utf-8')
reader = csv.reader(f)
for idx, line in enumerate(reader):
    if idx == 0:
        continue
        
    pair_data.append([line[0], line[1]])
f.close()

# pair data 확인
print(pair_data[:3])

[['12시 땡!', '하루가 또 가네요.'], ['1지망 학교 떨어졌어', '위로해 드립니다.'], ['3박4일 놀러가고 싶다', '여행은 언제나 좋죠.']]


### Step 3-2: Tokenization

기존의 `NLTK` 나 `SpaCy` 를 이용하여 **whitespace** 단위로 토큰화를 진행하게 되면, 한국어의 경우에는 성능이 저하됩니다. 이러한 성능 저하를 막기 위해, 한국어의 경우에는 형태소 분석기를 이용하여 토큰화를 진행합니다. 본 실습에서는 `Pynori` 형태소 분석기를 사용합니다.

- `KoreanAnalyzer`: argument 에 따른 한국어 형태소 분석기 초기화
- `filtering`: 형태소 분석기를 통해 나온 결과에서 **termAtt** & **posTagAtt** 추출

[github]: https://github.com/gritmind/python-nori
[블로그]: https://gritmind.github.io/2019/05/nori-deep-dive.html

Note: `Pynori` 관련 정보는 [github] 과 [블로그]에 자세히 작성되어 있습니다.

In [ ]:
# pynori 라이브러리 설치
!pip install pynori

In [ ]:
from pynori.korean_analyzer import KoreanAnalyzer

nori = KoreanAnalyzer(decompound_mode='DISCARD',
                      discard_punctuation=False,
                      output_unknown_unigrams=False,
                      pos_filter=False,
                      stop_tags=['JKS', 'JKB', 'VV', 'EF'])

In [ ]:
# 형태소 분석기를 통해 나온 결과 filtering
def filtering(result):
    text = result['termAtt']
    morp = result['posTagAtt']
    assert len(text) == len(morp)
    
    temp_list = list()
    for i in range(len(text)):
        if text[i] == ' ':
            continue

        temp_str = text[i] + '/' + morp[i]
        temp_list.append(temp_str)
    return temp_list

In [ ]:
# 형태소 분석된 [질문, 응답] 데이터셋 구축
total_data = list()
for each in pair_data:
    utter_result = nori.do_analysis(each[0])
    resp_result = nori.do_analysis(each[1])

    utter = filtering(utter_result)
    resp = filtering(resp_result)

    total_data.append([utter, resp])
    
# 데이터 사이즈 및 실제 결과 확인
print("Total size of data is", len(total_data))
print("\nExample:")
print(total_data[:1])

Total size of data is 11823

Example:
[[['12/SN', '시/NNBC', '땡/MAG', '!/SF'], ['하루/NNG', '가/JKS', '또/MAG', '가/VV', '네요/EF', './SF']]]


### Step 3-3: Data split & shuffling 

[링크]: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

학습 및 평가를 위해 원본 데이터에서 **90%** 는 학습 데이터로 사용하고, **10%** 는 평가 데이터로 사용합니다. 이 때, `sklearn` 라이브러리에 `train_test_split` 함수를 사용하고, 함수의 argument 정보들은 해당 [링크]로 가시면 확인할 수 있습니다.


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(total_data, test_size=0.1, random_state=42, shuffle=True)

# 학습 및 평가 데이터 크기 확인
print("Train/Test size is {}/{}".format(len(train), len(test)))
print("\nExample:")
print(train[:2])

Train/Test size is 10640/1183

Example:
[[['나/NP', '한테/JKB', '질리/VV', '면/EC', '어쩌/VV', '지/EC', '걱정/NNG', '되/XSV', '어/EC'], ['당신/NP', '의/JKG', '겉/NNG', '모습/NNG', '이/JKC', '아니/VCN', 'ᆫ/ETM', '진정/XR', '하/XSA', 'ᆫ/ETM', '내면/NNG', '의/JKG', '모습/NNG', '을/JKO', '보이/VV', '어/EC', '주/VX', '시/EP', '어요/EF', './SF']], [['새롭/VA', 'ᆫ/ETM', '일/NNG', '벌리/VV', 'ㅓ도/EC', '되/VV', 'ᆯ까/EC'], ['도전/NNG', '하/XSV', '아/EC', '보/VX', '아도/EC', '좋/VA', '을/ETM', '거/NNB', '같/VA', '아요/EF', './SF']]]


### Step 3-4: Create Word Dictionary

실제 자연어로 이루어진 단어들을 기계가 이해할 수 있는 index 값으로 맵핑해주는 dictionary를 구축합니다. 추가로, 응답을 생성하는 단계를 위해 index 가 단어로 맵핑되는 dictionary 도 구축합니다.

- `build_dict`: 학습 데이터의 토큰들을 빈도수 단위로 내림차순 정렬하여, 위에서부터 **threshold** 기준으로 dictionary 갯수/사이즈를 지정
    - special tokens
        - **pad**: GPU를 이용하여 모델을 학습시키기 위해서는 batch 내에 있는 모든 문장들이 동일한 길이를 가져야한다. 이를 위해, maximum 길이를 지정하고 남은 부분은 padding token 을 채워줍니다. 주의할 점은, 학습시 loss 를 구할 때 padding 에 해당되는 부분은 반영시키지 않아야합니다.
        - **unk**: word dictionary 에 없는 단어(token)가 등장하면 unknown token 을 채워줍니다.
        - **sos**: 디코더에서 문장의 시작을 알리는 토큰입니다.
        - **eos**: 디코더에서 문장의 끝을 알리는 토큰입니다.

In [ ]:
def build_dict(data, threshold=40000):
    
    if not os.path.exists(WORD_DICT_DIR):
        """
        Build word dictionary
        """
        
        vocab = list()
        for doc in data:
            for word in doc[0]:
                vocab.append(word)
            for word in doc[1]:
                vocab.append(word)
        
        counter = collections.Counter(vocab).most_common(threshold)
        
        word2idx = dict()
        word2idx['<pad>'] = 0
        word2idx['<unk>'] = 1
        word2idx['<sos>'] = 2
        word2idx['<eos>'] = 3
        
        for word, _ in counter:
            word2idx[word] = len(word2idx)
        
        with open(WORD_DICT_DIR, 'wb') as f:
            pc.dump(word2idx, f)
    else:
        """
        Load word dictionary which was built before
        """
        with open(WORD_DICT_DIR, 'rb') as f:
            word2idx = pc.load(f)
    
    print("Load word dictionary and vocab")
    return word2idx

In [ ]:
# build_word2idx
word2idx = build_dict(train, THRESHOLD)
idx2word = {idx: word for word, idx in word2idx.items()}

vocab_size = len(word2idx)

print("Size of word2idx is {}".format(len(word2idx)))

Load word dictionary and vocab
Size of word2idx is 5652


## Step 4: Prepare data for Model

모델의 학습을 위해 학습 데이터를 word dictionary 를 이용하여 index 로 변환해줍니다. 그리고 **mini-batch stochastic gradient descent** 를 위해 학습 데이터를 mini-batch 단위로 준비해줍니다.

- `batch_iter`: 학습 과정에서 iteration 돌 때마다, 배치 단위의 데이터를 불러오는 과정을 위한 함수
- `batch_dataset`: mini-batch 단위 학습 데이터 만들기 위한 함수
    - 모델 학습의 efficiency 를 위해 **MAX_LEN** 만큼 데이터 자르기
    - 처음 보는 단어는 **unk** 토큰으로 채우기
    - utterance 문장의 뒤에 **eos** 토큰 더하기
    - response 문장의 앞/뒤에 **sos/eos** 토큰 더하기
    - **MAX_LEN** 까지 남은 부분은 **pad** 토큰 채우기

In [ ]:
def batch_iter(data, batch_size):
    #num_batches_per_epoch = (len(data) - 1) // batch_size + 1
    num_batches_per_epoch = int(len(data) / batch_size)
    data = np.array(data)
    
    for batch_idx in range(num_batches_per_epoch):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(data))
        enc_data = list()
        dec_data = list()
        
        for each in data[start_idx:end_idx]:
            enc_data.append(each[0])
            dec_data.append(each[1])

        yield enc_data, dec_data
        
def batch_dataset(batch_x, batch_y, word2idx):
    # batch input & target
    batch_x = list(map(lambda x: x[:MAX_LEN], batch_x))
    batch_y = list(map(lambda x: x[:MAX_LEN], batch_y))

    batch_x = list(map(lambda x: [word2idx.get(each, word2idx['<unk>']) for each in x], batch_x))
    batch_y = list(map(lambda x: [word2idx.get(each, word2idx['<unk>']) for each in x], batch_y))
                        
    batch_enc_input = list(map(lambda x: list(x) + [word2idx['<eos>']], batch_x))            
    batch_dec_target = list(map(lambda x: [word2idx['<sos>']] + list(x) + [word2idx['<eos>']], batch_y))
            
    batch_enc_input = list(map(lambda x: list(x) + (MAX_LEN+1 - len(x)) * [word2idx['<pad>']], batch_enc_input))         
    batch_dec_target = list(map(lambda x: list(x) + (MAX_LEN+2 - len(x)) * [word2idx['<pad>']], batch_dec_target))
    
    batch_enc_input = np.array(batch_enc_input)
    batch_dec_target = np.array(batch_dec_target)
    
    return batch_enc_input, batch_dec_target

## Step 5: Construct the seq2seq model with attention mechanism

[Google Neural Machine Translation]: https://arxiv.org/abs/1609.08144

인코더-디코더 구조의 가장 대표적인 seq2seq 모델은 두 개의 recurrent neural network 로 구성된 구조로써, 입력 문장에 대해 상응하는 출력 문장을 생성하게 이루어져있습니다. 또한 seq2seq 모델의 경우에는 가변적으로 변하는 문장의 길이를 잘 처리할 수 있다는 장점이 있습니다. 그렇기 때문에, 오늘날 기계 번역 및 대화 생성 등 여러 분야에서 사용되고 있습니다. 아래 그림은 [Google Neural Machine Translation] 모델입니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/gmt.png?raw=True" width="80%" height="60%" title="gmt" alt="gmt"></img></center>

Seq2Seq 모델은 크게 두가지 부분으로 구성되어있습니다. 
- Encoder: **Sentence representation**
    - 입력 문장을 하나의 고정된 차원의 context vector 로 변환
    - 각 time step 에서 output 은 context dependent vector representation
- Decoder: **Language modeling**
    - context vector 가 주어졌을 때, 입력 문장에 적합한 출력 문장을 생성
    - Conditional language modeling 이므로, causal structure 유지
    
<center><img src="https://github.com/passing2961/KEMC/blob/master/seq2seq.png?raw=True" width="80%" height="60%" title="seq2seq" alt="seq2seq"></img></center>

*이미지 출처: https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#definition*

[Bahdanau et al.]: https://arxiv.org/abs/1409.0473

그러나, seq2seq 모델의 경우에는 문제점이 존재합니다. 인코더에서 **입력 문장의 문맥적 정보를 하나의 고정된 차원의 벡터에 다 담으려고 (collapsing)** 하다보니, 입력 문장의 길이가 길어지게 되면 **정보 손실 (information loss)** 이 발생합니다. 결국, 해당 문제는 **long-term dependency** 가 보장이 안된다는 것인데, 이를 위해 attention mechanism 이 **[Bahdanau et al.]** 에 의해 처음으로 제안되었습니다. 

Attention mechanism 은 long-term dependency 도 잘 보장하면서 동시에 디코더에서 다음 단어를 생성할 때 **입력 문장에서 제일 중요하게 반영해야하는 단어**를 반영할 수 있게 해줍니다. 즉, 기존의 seq2seq 에서 하나의 context vector 만 사용하는 것과는 달리, attention mechanism 을 적용하게 되면 디코더의 각 time step 마다 **dynamic 하게 context vector 를 사용할 수 있다는 장점**이 있습니다.

[Luong]: https://arxiv.org/abs/1508.04025

앞서 말하였지만, **Bahdanau** attention model 을 시작으로 많은 어텐션 모델들이 등장하였습니다. 그 중에, **Bahdanau** 어텐션 모델을 효율적이고 좀 더 simple 하게 만든 **[Luong]** 어텐션 모델을 본 실습에서는 다루겠습니다. (*Transfomer 모델 나오기 전까지 가장 많이 쓰였던 어텐션 모델입니다.*) 그 전에, luong 어텐션 메커니즘이 어떻게 동작이 이루어지는 아래의 그림을 보고 확인해보겠습니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/seq2seq_attn.jpeg?raw=true" width="80%" height="70%" title="seq2seq_attn" alt="seq2seq_attn"></img></center>

*이미지 출처: https://towardsdatascience.com/the-definitive-guide-to-bidaf-part-3-attention-92352bbdcb07*

먼저, 디코더에서 현재 time step 의 RNN hidden state 값이랑 인코더에 있는 모든 RNN hidden state 들이랑 dot product (matrix multiplication) 연산을 하여, 디코더의 현재 time step 값의 단어가 인코더의 어느 단어와 제일 관련이 있는지를 구합니다. 이렇게 구해진 벡터를 **attention score** 라고 부릅니다. 여기서 dot product 는 보통 **score function** 이라고 말합니다. 이렇게 score function 을 통해 구한 attention score 벡터 값을 softmax 함수를 통과시켜 attention weight 을 구합니다.
(*softmax 함수는 입력이 주어졌을 때, 출력값이 확률이 되는 형태가 되도록 해줍니다.*)

<center><img src="https://github.com/passing2961/KEMC/blob/master/softmax.png?raw=true" width="50%" height="40%" title="softmax" alt="softmax"></img></center>

*이미지 출처: https://medium.com/data-science-bootcamp/understand-the-softmax-function-in-minutes-f3a59641e86d*

그런 다음, attention weight 과 인코더의 모든 RNN hidden state 들의 값을 weighted sum 연산을 통해 하나의 **context vector** 를 구합니다. 그리고 현재 time step 에서의 단어(토큰)을 예측할 때, 디코더의 현재 time step 에서의 hidden state 값(output)과 구한 context vector 를 결합(concatenation)하여 softmax 함수를 통과시킵니다.

대표적인 어텐션 모델인 **Bahdanau** 와 **Luong** 의 계산 과정을 수식을 통해 자세하게 확인해보겠습니다.

**Bahdanau attention mechanism**

<center><img src="https://github.com/passing2961/KEMC/blob/master/bah_attn.png?raw=true" width="30%" height="20%" title="bah_attn" alt="bah_attn"></img></center>

Bahdanau attention mechanism 계산 과정

- score function(additive style)

<center><img src="https://github.com/passing2961/KEMC/blob/master/bah_score.png?raw=true" width="30%" height="20%" title="bah_score" alt="bah_score"></img></center>

- attention weights

<center><img src="https://github.com/passing2961/KEMC/blob/master/bah_attn_weight.png?raw=true" width="30%" height="20%" title="bah_attn_weight" alt="bah_attn_weight"></img></center>

- context vector

<center><img src="https://github.com/passing2961/KEMC/blob/master/bah_context.png?raw=true" width="20%" height="10%" title="bah_context" alt="bah_context"></img></center>

- prediction: a deep-output and a maxout layer



**Luong attention mechanism**

<center><img src="https://github.com/passing2961/KEMC/blob/master/luong_attn.png?raw=true" width="100%" height="100%" title="luong_attn" alt="luong_attn"></img></center>

Luong attention mechanism (Global) 계산 과정

- score function

<center><img src="https://github.com/passing2961/KEMC/blob/master/luong_score.png?raw=true" width="70%" height="60%" title="luong_score" alt="luong_score"></img></center>

- attention weights

<center><img src="https://github.com/passing2961/KEMC/blob/master/luong_attn_weight.png?raw=true" width="40%" height="30%" title="luong_attn_weight" alt="luong_attn_weight"></img></center>

- context vector

<center><img src="https://github.com/passing2961/KEMC/blob/master/luong_context.png?raw=true" width="20%" height="10%" title="luong_context" alt="luong_context"></img></center>

- predictions

<center><img src="https://github.com/passing2961/KEMC/blob/master/luong_concat.png?raw=true" width="30%" height="20%" title="luong_concat" alt="luong_concat"></img></center>

<center><img src="https://github.com/passing2961/KEMC/blob/master/luong_softmax.png?raw=true" width="35%" height="25%" title="luong_softmax" alt="luong_softmax"></img></center>


In [ ]:
# Encoder Model
class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        
        # Initialize LSTM with bidirectional
        # the input_size and hidden_size params are both set to 'hidden_size'
        # because our input size is a word embedding with number of features == hidden_size
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, bidirectional=True, dropout=dropout, batch_first=True)
            
        ##########################################################################
        #### TODO: Initialize GRU with bidirectional                          ####
        ##########################################################################

        # self.gru = 

        ##########################################################################

    def forward(self, enc_input):
        # Convert word indexed to embeddings (mapping discrete tokens to continuous space)
        # embedded shape == (batch_size, enc_max_len, embed_size)
        embedded = self.embedding(enc_input)
       
        # Forward pass through RNN module
        # if bidirectional, outputs shape == (batch_size, enc_max_len, hidden_size*2)
        # if not bidirectional, outputs shape == (batch_size, enc_max_len, hidden_size)
        # hidden shape == (num_directions * num_layers, batch_size, hidden_size)
        outputs, (hidden, cell) = self.lstm(embedded)

        ##########################################################################
        #### TODO: Forward pass through GRU module                            ####
        ####       Return value should be changed.                            ####
        ##########################################################################

        # outputs, hidden = 

        ##########################################################################

        # Sum bidirectional outputs
        # shape == (batch_size, enc_max_len, hidden_size)
        # outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
        
        # output of shape (seq_len, batch, num_directions * hidden_size)
        # h of shape (num_layers * num_directions, batch, hidden_size)
        # c of shape (num_layers * num_directions, batch, hidden_size)
        return outputs, (hidden, cell)

In [ ]:
# Luong attention layer
class Attention(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attention, self).__init__()
        self.method = method
        
        # method: indicator that determines the score function 

        # raise: 프로그래머가 지정한 예외가 발생할 수 있도록 강제함
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        
        self.hidden_size = hidden_size   
        ##########################################################################
        #### TODO: Initialize network for 'general' & 'concat' score function ####
        ####                                                                  ####
        ##########################################################################     
    
    def dot_score(self, hidden, encoder_output):        
        # attention dot score function (Luong)
        # attention score shape == (batch_size, dec_max_len)
        return torch.sum(hidden * encoder_output, dim=2)
    
    def general_score(self, hidden, encoder_output):
        ##########################################################################
        #### TODO: Define the 'general' score function                          ####
        ####                                                                  ####
        ########################################################################## 
        return

    def concat_score(self, hidden, encoder_output):
        ##########################################################################
        #### TODO: Define the 'concat' score function                         ####
        ####                                                                  ####
        ########################################################################## 
        return

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'dot':
            attn_weights = self.dot_score(hidden, encoder_outputs)
        
        ##########################################################################
        #### TODO: Calculate the attention weights                            ####
        ####       based on 'general' and 'concat' method                     ####
        ########################################################################## 

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_weights, dim=1).unsqueeze(1)

In [ ]:
# Decoder Model
class AttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, vocab_size, n_layers=1, dropout=0.1):
        super(AttnDecoderRNN, self).__init__()
        
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        # Define layers
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        #self.embedding_dropout = nn.Dropout(dropout)
        
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, bidirectional=False, num_layers=1, batch_first=True)
        self.concat = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.vocab_size)

        ##########################################################################
        #### TODO: Initialize GRU with unidirectional                         ####
        ##########################################################################

        # self.gru = 

        ##########################################################################

        self.attn = Attention(self.attn_model, self.hidden_size)
    
    def forward(self, dec_input, hidden, cell, encoder_outputs):
        # Note: we run this one step (word) at a time
        
        # Get embedding of current input word
        # embedded shape == (batch_size, 1, embed_size)
        embedded = self.embedding(dec_input)
    
        # Forward through unidirectional LSTM
        # output shape == (batch_size, 1, hidden_size)
        # hidden shape == (num_directions * num_layers, batch_size, hidden_size)        
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        ##########################################################################
        #### TODO: Forward through unidirectional LSTM                        ####
        ####       Return values should be changed.                           ####
        ##########################################################################

        # output, hidden = 

        ##########################################################################

        # Calculate attention weights from the current GRU output
        # attention weights shape == (batch_size, 1, dec_max_len)
        attn_weights = self.attn(output, encoder_outputs)
        
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        # context vector shape == (batch_size, 1, hidden_size)
        context = attn_weights.bmm(encoder_outputs)
        
        # Concatenate weighted context vector and GRU output using Luong
        # output shape == (batch_size, hidden_size)
        # context shape == (batch_size, hidden_size)
        # concat_input shape == (batch_size, hidden_size * 2)
        # concat_output shape == (batch_size, hidden_size)
        output = output.squeeze(1)
        context = context.squeeze(1)
        
        concat_input = torch.cat((output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        
        # Predict next word using Luong
        # output shape == (batch_size, vocab_size)
        output = self.out(concat_output)
        output = F.log_softmax(output, dim=1)
        
        # Return output and final hidden state
        return output, hidden, cell

## Step 5: Define the optimizer and the loss function

- optimizer: Adam 사용
- loss function: negative log likelihood 사용

In [ ]:
# Initialize encoder & decoder models
encoder = EncoderRNN(vocab_size, enc_hidden_size, encoder_n_layers, dropout).cuda()
decoder = AttnDecoderRNN(attn_model, dec_hidden_size, vocab_size, decoder_n_layers, dropout).cuda()

print("Models build and ready to go!")

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Models build and ready to go!


In [ ]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

criterion= nn.NLLLoss(ignore_index=word2idx['<pad>'])
#criterion= nn.NLLLoss()

## Step 6: Training loop

**The goal of AI**

<center><img src="https://github.com/passing2961/KEMC/blob/master/basic1.png?raw=true" width="80%" height="70%"  title="basic1" alt="basic1"></img></center>

<center><img src="https://github.com/passing2961/KEMC/blob/master/basic2.png?raw=true" width="80%" height="70%"  title="basic2" alt="basic2"></img></center>

**Pipeline of training machine learning systems**

- **Collect a data**: training/validation/test
  - training: 실제 학습 용도
  - validation: overfitting 방지 용도
  - test: 실제 평가 용도

- **Hypothesis set**: Design a network architecture

- **Loss function**: Define a loss (objective) function

- **optimize loss function**
  - Automatic backpropagation: Compute the minibatch gradient
  - Optimization: Update the parameters (e.g. Stochastic Gradient Descent)
  - Earlystopping: An efficient way to prevent overfitting
  - Adaptive learning rate

위의 과정을 자세히 살펴보겠습니다.

#### **Collect a data**

먼저, 학습시키기 위해서는 데이터가 필요합니다. 이를 위해 데이터를 수집합니다. 이 경우에 직접 크롤링을 통해 데이터를 수집할 수도 있지만, 보통은 benchmark 처럼 기존에 표준처럼 사용되는 데이터를 사용합니다. 이렇게 수집한 이후에는 데이터를 train/validation/test 3개의 데이터셋으로 나눕니다. Training dataset 은 실제 모델의 학습을 위해 필요합니다. Valdiation dataset 은 검증 데이터셋으로써, 모델이 학습 도중에 너무 학습 데이터만 기억하게 되는 overfitting (memorization) 이 일어나는 것을 방지하기 위해 필요합니다. Test dataset 은 실제 학습된 모델의 성능을 평가하기 위해 필요합니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/batch_data.png?raw=true" width="80%" height="70%"  title="batch_data" alt="batch_data"></img></center>

- 입력 인풋 형태: (batch_size, max_len)

#### **Hypothesis set**

데이터를 수집한 이후에는 주어진 task 에 대해서 어떤 모델 (neural network) 를 사용 혹은 고안할지 결정합니다. 예를 들어, 이미지가 고양이인지 강아지인지 분류하는 문제에서는 hypothesis set 은 VGGnet, GoogLeNet, ResNet 등이 포함될 수 있습니다. 즉, hypothesis set 은 무한 (infinite) 합니다.

#### **Loss function**

어떤 모델을 사용할지 결정하였으면, 해당 모델에서 loss function 을 설계합니다. 예를 들어, classification 문제의 경우에는 hinge loss, log loss 등이 있고, regression 문제의 경우에는 mean squared error (MSE), mean absolute error, robust loss 등이 있습니다. 그러나 대부분의 딥러닝 모델들은 **distribution-based loss function** 을 사용합니다. Distribution-based loss function 의 distribution 은 입력 x 가 주어졌을 때, 출력 y 가 특정 y' 값일 경우가 얼마나 가능한지 (혹은 그럴듯한지) 를 확인합니다. 

<center><img src="https://github.com/passing2961/KEMC/blob/master/dist_loss.png?raw=true" width="40%" height="30%"  title="dist_loss" alt="dist_loss"></img></center>
    
위의 수식이 distribution-based loss function 의 distribution 에 해당합니다. 그렇다면, 등식의 우변항이 어떤 종류의 문제이냐에 따라 distribution 이 달라집니다.

- Binary classification: Bernoulli distribution
- Multiclass classification: Categorical distribution
- Linear regression: Gaussian distribution
- Multimodal linear regression: Mixture of Gaussians

이렇게 distribution 을 정하고 나면, 위의 conditional probability 수식이 training data 에 대해서 **maximally likely** 해야합니다. 아래의 수식에 해당합니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/log_prob.png?raw=true" width="50%" height="40%"  title="log_prob" alt="log_prob"></img></center>

위의 수식이 **log-likelihood** 라고 부르고, 해당 수식을 최대화 되게끔 학습이 이루어집니다. 그런데 Loss function 은 최소화되어야하므로 아래의 수식처럼 log-likelihood 앞에 (-) 를 붙입니다. 이를 **negative log-likelihood** 라고 부르고, distribution-based loss function 이 됩니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/nll.png?raw=true" width="40%" height="30%"  title="nll" alt="nll"></img></center>

#### **optimizer loss function**

이렇게 loss function 을 결정한 이후에, 해당 loss 값을 최소화시키기 위해 어떻게 해야할까요? 고등수학으로 돌아가보면, 미분가능한 함수에서 최솟값은 미분값이 0인 지점이라는 것을 알 수 있습니다. 이 원리를 이용해서, gradient-based optimization 방법이 제안되었습니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/gradient.png?raw=true" width="60%" height="50%"  title="gradient" alt="gradient"></img></center>

Gradient-based optimization 알고리즘을 살펴보면 아래와 같습니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/gradient_alg.png?raw=true" width="60%" height="50%"  title="gradient_alg" alt="gradient_alg"></img></center>

그러나 해당 방법은 **모든 데이터에 대해서 한번에 gradient 를 계산해야하므로 비용이 많이 든다**는 문제점이 있습니다. 이를 해결하기 위해, 전체 데이터 셋에서 랜덤하게 하나만 샘플링해서 gradient 를 계산하는 방법인 **stochastic gradient descent** 방법이 제안되었습니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/stochastic_alg.png?raw=true" width="60%" height="50%"  title="stochastic_alg" alt="stochastic_alg"></img></center>

위의 방법은 gradient-based optimization 방법보다는 효율적으로 계산이 가능하지만, **굉장히 noisy 하다**는 문제점이 있습니다. 만약 데이터 하나가 전체 데이터셋을 나타내는 표본이 아닌 경우에는, noise 가 있는 데이터 하나를 샘플링하게 되는 것이고 그렇게 되면 optimization 자체는 noise 가 있을 수 밖에 없습니다. 따라서, 데이터 하나를 샘플링하는 것이 아니라 묶음 단위로 하는 방법인 **minibatch stochastic gradient descent** 방법이 제안되었습니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/mini_stochastic.png?raw=true" width="60%" height="50%"  title="mini_stochastic" alt="mini_stochastic"></img></center>

이렇게 모델의 학습을 진행하면 학습 모델의 loss 값이 계속 낮아지도록 학습이 이루어집니다. 그런데, 모델의 training loss 값이 낮은게 정말 좋은 모델을 의미할까요?

<center><img src="https://github.com/passing2961/KEMC/blob/master/overfitting.png?raw=true" width="70%" height="60%"  title="overfitting" alt="overfitting"></img></center>

위의 그림을 보시면, 오른쪽 모델의 training loss 가 제일 낮은 것을 확인할 수 있습니다. 그러나 해당 모델이 제일 좋다고는 말할 수 없습니다. 왜냐하면 모델이 학습 데이터에 대해서는 좋은 성능을 보일 수 있어도, 너무 학습 데이터에만 과하게 학습이 되어서 새로운 (처음보는) 데이터가 들어왔을 때는 성능이 좋지 않을 수 있습니다. 이런 문제를 **overfitting** 이라고 부릅니다. Overfitting 현상은 모델이 학습 데이터를 일종의 **"memorize"** 현상이라고 볼 수 있습니다. **보통 training loss 는 낮은데, test error 는 높아질 때 발생합니다.** 또한, 모델의 parameter 가 training data 수보다 많은 경우에 일반적으로 발생합니다.

이를 해결하는 방법 (Regularization) 이 대표적으로 **weight decay, dropout, early stopping** 이 있습니다. 

- **weigth decay**: Penalize complex solution using **additional constraints**

<center><img src="https://github.com/passing2961/KEMC/blob/master/weight_decay.png?raw=true" width="70%" height="60%"  title="weight_decay" alt="weight_decay"></img></center>

- **Dropout**: **Randomly turn off** activations with some probability p

<center><img src="https://github.com/passing2961/KEMC/blob/master/dropout.png?raw=true" width="70%" height="60%"  title="dropout" alt="dropout"></img></center>

- **Early stopping**: Stop training once the validation loss starts to increase

<center><img src="https://github.com/passing2961/KEMC/blob/master/earlystopping.png?raw=true" width="70%" height="60%"  title="earlystopping" alt="earlystopping"></img></center>

*이미지 출처: MIT 6.S191: Introduction to Deep Learning*

이렇게 Regularization 의 대표적인 3가지 기법들을 알아보았습니다. 그렇다면, 실제로 minibatch stochastic gradient descent 가 어떻게 이루어질까요?

1. random 하게 batch 단위의 데이터를 불러오기
2. minibatch gradient 계산
3. 모델의 parameter 들 업데이트
4. validation loss 가 더이상 좋아지지 않을 때까지 1, 2, 3 과정 반복

[Adam optimizer]: https://arxiv.org/abs/1412.6980

stochastic gradient descent 방법은 learning rate 를 고정시키고 학습을 진행하는데, 고정시킨 learning rate 값이 모델의 학습을 불안정 시키게 할 수 있습니다. 이를 위해 adaptive 하게 learning rate 를 조절하는 optimization 방법이 제안되었습니다. 대표적으로 **[Adam optimizer]** 가 있습니다. 현재 제일 많이 사용되는 optimizer 입니다. 본 실습에서도 Adam optimizer 를 사용할 것입니다. 아래는 optimizer 관련 계보입니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/optimizers.png?raw=true" width="70%" height="60%"  title="optimizers" alt="optimizers"></img></center>



#### **본 실습에서 학습 과정**

- `batch_iter` 를 통해 encoder 입력 문장과 decoder 입력 문장을 배치사이즈 단위로 가져오기
- Forward pass: 
  - Encoder 에 입력 문장 주기
  - Decoder 에 step-by-step 으로 단어 토큰 예측/생성
  - Teacher forcing 적용
- Backward pass:
  - minibatch gradient 계산 (backpropagation)
  - 모델의 파라미터들 업데이트 (optimization)
  
#### **Teacher Forcing**

**만약에 디코더에서 첫 토큰이 잘못 예측되면, 그 뒤에 생성되는 토큰들도 잘못된 방향으로 가지 않을까?** (*autoregressive 속성 때문*)

<center><img src="https://github.com/passing2961/KEMC/blob/master/autoregressive.png?raw=true" width="60%" height="50%"  title="autoregressive" alt="autoregressive"></img></center>

이런 문제를 해결하기 위해 teacher forcing 이라는 기법이 등장하였습니다. Teacher forcing 의 정의는 "**the technique where the target word is passed as the next input to the decoder**" 입니다. 즉, 학습과정에서 실제 정답 문장이 입력으로 주어지는 것을 말합니다. 

- 장점
  - teacher forcing 기법으로 학습을 진행하면 수렴 (convergence) 이 빨리 진행됩니다.
- 단점
  - training 과정이랑 inference 과정이 다르기 때문에 실제 성능이 낮을 수 있습니다. (*Exposure bias*)

<center><img src="https://github.com/passing2961/KEMC/blob/master/teacher_forcing.png?raw=true" width="40%" height="30%"  title="teacher_forcing" alt="teacher_forcing"></img></center>

##### **참고: Gradient Clipping**

<center><img src="https://github.com/passing2961/KEMC/blob/master/clipping.png?raw=true" width="60%" height="50%"  title="clipping" alt="clipping"></img></center>

*이미지 출처: Goodfellow et al. Deep Learning. 2016 https://www.deeplearningbook.org/contents/rnn.html*

학습 도중에 gradient 가 너무 커지는 현상인 "**exploding gradient**" 문제를 해결하는 방법입니다. 쉽게 설명드리자면, gradient 크기가 일정 기준(threshold)이상 너무 커지면 gradient 크기를 재조정하는 것입니다.

<center><img src="https://github.com/passing2961/KEMC/blob/master/gradient_clipping_alg.png?raw=true" width="60%" height="50%"  title="clipping_alg" alt="clipping_alg"></img></center>

*Note: gradient clipping 관련 논문: https://arxiv.org/abs/1211.5063*

##### **참고: Perplexity**

Perplexity 는 context 를 봤을 때, 그 다음 단어를 몇 개의 vocabulary subset 안에서 고를 수 있는지 알려주는 지표입니다.

- ppl=1 => 입력을 보면 완벽하게 다음 단어를 예측할 수 있다
- ppl=10 => 입력을 보면 다음 단어가 무엇일지 10개 내외에서 맞출 수 있다
- ppl=|V| => 아무리 입력을 봐도 대체 다음에 뭐가 나올지 맞출 수 없는 상황이다

<center><img src="https://github.com/passing2961/KEMC/blob/master/ppl.png?raw=true" width="60%" height="50%"  title="ppl" alt="ppl"></img></center>


In [ ]:
class EarlyStopping(object):
    """
    EarlyStopping
    """
    def __init__(self, best_loss, max_patience):
        super(EarlyStopping, self).__init__()
        
        self.best_loss = best_loss
        self.patience = 0
        self.max_patience = max_patience
        self.best_epoch = 0
        
    def update(self, recent_loss, recent_epoch):
        if self.best_loss > recent_loss:
            self.best_loss = recent_loss
            self.patience = 0
            
            self.best_epoch = recent_epoch
            return "update"
        else:
            self.patience += 1
            if self.patience == self.max_patience:
                return "terminate"
            
            return "patience"

In [ ]:
def evaluate(encoder, decoder, val_batches, device, word2idx):

    
    total_loss = 0.
    
    for batch_idx, (batch_x, batch_y) in enumerate(val_batches):
        if batch_idx == 1:
            break
            
        batch_enc_input, batch_dec_target = batch_dataset(batch_x, batch_y, word2idx)
        
        batch_enc_input = torch.tensor(batch_enc_input, dtype = torch.long, device='cuda')
        batch_dec_target = torch.tensor(batch_dec_target, dtype = torch.long, device='cuda')
        
        loss = 0.
        
        with torch.no_grad():
            # Forward pass through encoder
            encoder_outputs, (encoder_hidden, encoder_cell) = encoder(batch_enc_input)
            #encoder_output, (eh, ec) = encoder(enc_input)

            eh = torch.cat((encoder_hidden[0], encoder_hidden[1]), dim=1).unsqueeze(0)
            ec = torch.cat((encoder_cell[0], encoder_cell[1]), dim=1).unsqueeze(0)
            
            # Create initial decoder input (start with <sos> tokens for each sentence)
            dec_input = torch.tensor([word2idx['<sos>']] * batch_size, dtype=torch.long, device=device)
            dec_input = dec_input.unsqueeze(1)

            # Set initial decoder hidden state to the encoder's final hidden state
            decoder_hidden = encoder_hidden[:decoder.n_layers]
            
            for t in range(1, batch_dec_target.size(1)):
                decoder_output, eh, ec = decoder(dec_input, eh, ec, encoder_outputs)

                # Calculate and accumulate loss
                loss += criterion(decoder_output, batch_dec_target[:, t])

                # Teacher forcing: next input is current target
                dec_input = batch_dec_target[:, t].unsqueeze(1)
                
            batch_loss = loss.item() / int(batch_dec_target.size(1))
            total_loss += batch_loss
        
    return total_loss / (batch_idx + 1)    

In [ ]:
%%time

# Define Earlystopping class
earlystopping = EarlyStopping(best_loss, max_patience)

num_batches_per_epoch = int(len(train) / batch_size)
print("[num_batches_per_epoch] {}".format(num_batches_per_epoch))

    
for epoch in range(max_epochs):

    total_loss = 0.
    total_ppl = 0.
    print_loss = 0.
    
    # Load batches for iteration
    train_batches = batch_iter(train, batch_size)
    
    encoder.train()
    decoder.train()

    # Training loop
    for batch_idx, (batch_x, batch_y) in enumerate(train_batches):

        batch_enc_input, batch_dec_target = batch_dataset(batch_x, batch_y, word2idx)
        
        # Initialize variables
        loss = 0.
        
        # zero gradients
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        batch_enc_input = torch.tensor(batch_enc_input, dtype = torch.long, device='cuda')
        batch_dec_target = torch.tensor(batch_dec_target, dtype = torch.long, device='cuda')

        # Forward pass through encoder
        encoder_outputs, (encoder_hidden, encoder_cell) = encoder(batch_enc_input)
                 
        eh = torch.cat((encoder_hidden[0], encoder_hidden[1]), dim=1).unsqueeze(0)
        ec = torch.cat((encoder_cell[0], encoder_cell[1]), dim=1).unsqueeze(0)
        
        # Create initial decoder input (start with <sos> tokens for each sentence)
        dec_input = torch.tensor([word2idx['<sos>']] * batch_size, dtype=torch.long, device=device)
        dec_input = dec_input.unsqueeze(1)

        # Set initial decoder hidden state to the encoder's final hidden state
        decoder_hidden = encoder_hidden[:decoder.n_layers]

        # Determine if we are using teacher forcing this iteration
        # random.random() -> 0~1 사이의 난수 생성
        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
        
        if use_teacher_forcing:
            for t in range(1, batch_dec_target.size(1)):
                
                decoder_output, eh, ec = decoder(dec_input, eh, ec, encoder_outputs)
                
                # Calculate and accumulate loss
                loss += criterion(decoder_output, batch_dec_target[:, t])

                # Teacher forcing: next input is current target
                dec_input = batch_dec_target[:, t].unsqueeze(1)                
        else:
            for t in range(1, batch_dec_target.size(1)):
                decoder_output, eh, ec = decoder(dec_input, eh, ec, encoder_outputs)
            
                # No teacher forcing: next input is decoder's own current output
                _, topi = decoder_output.topk(1)
            
                dec_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
                dec_input = decoder_input.to(device)
            
                # Calculate and accumulate loss
                loss += criterion(decoder_ouput, batch_dec_target[:, t])
            
            
        batch_loss = (loss.item() / int(batch_dec_target.size(1)))
        batch_ppl = np.exp(batch_loss)
        
        total_loss += batch_loss
        total_ppl += batch_ppl
        print_loss += loss.item()
        
        # Perform backpropagation
        loss.backward()
        
        # gradient clipping: gradients are modified in place
        #_ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
        #_ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
        
        # Adjust/Update model weights
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        if (batch_idx + 1) % LOG_INTERVAL == 0:
            print("[epoch {} | step {}/{}] loss: {:.4f} (Avg. {:4f}) PPL: {:.4f} (Avg. {:.4f})".format(epoch+1,
                                                                                                       batch_idx+1,
                                                                                                       num_batches_per_epoch,
                                                                                                       batch_loss, total_loss/(batch_idx + 1),
                                                                                                       batch_ppl, total_ppl/(batch_idx + 1)))

    # How to use EarlyStopping
    val_batches = batch_iter(test, batch_size)
    

    encoder.cuda().eval()
    decoder.cuda().eval()
    val_loss = evaluate(encoder, decoder, val_batches, device, word2idx)
    print("[epoch {}] loss: {:.4f}".format(epoch+1, val_loss))    
    
    # EarlyStopping for preventing overfitting problem
    exitcode = earlystopping.update(val_loss, epoch)
    if exitcode == 'update':
        # 학습된 모델 저장
        torch.save(encoder, dirpath + 'best_encoder_' + str(epoch + 1) + '.pt')
        torch.save(decoder, dirpath + 'best_decoder_' + str(epoch + 1) + '.pt')
        print("[epoch {}] patience: {}\tmax_patience: {}\tbest_loss: {}\tModel best performance!".format(epoch+1, 
                                                                                                         earlystopping.patience,
                                                                                                         earlystopping.max_patience,
                                                                                                         earlystopping.best_loss))
    
    
    torch.save(encoder, dirpath + 'encoder_' + str(epoch + 1) + '.pt')
    torch.save(decoder, dirpath + 'decoder_' + str(epoch + 1) + '.pt')
    #total_loss_per_epoch = (total_loss / (batch_idx + 1))
            

[num_batches_per_epoch] 332
[epoch 1 | step 100/332] loss: 0.1151 (Avg. 0.121841) PPL: 1.1220 (Avg. 1.1298)
[epoch 1 | step 200/332] loss: 0.1194 (Avg. 0.116804) PPL: 1.1268 (Avg. 1.1241)
[epoch 1 | step 300/332] loss: 0.0925 (Avg. 0.111069) PPL: 1.0969 (Avg. 1.1177)
[epoch 1] loss: 0.7992
[epoch 1] patience: 0	max_patience: 5	best_loss: 0.7992199085376881	Model best performance!


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type EncoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type AttnDecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[epoch 2 | step 100/332] loss: 0.0789 (Avg. 0.086322) PPL: 1.0821 (Avg. 1.0903)
[epoch 2 | step 200/332] loss: 0.1079 (Avg. 0.087765) PPL: 1.1139 (Avg. 1.0919)


KeyboardInterrupt: ignored